In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from qutip import *

### Introduction

The Hamiltonian for a Transmon qubit is

$\displaystyle H = \sum_n 4 E_C (n_g - n)^2 \left|n\right\rangle\left\langle n\right| - E_{J\Sigma} [\cos(\pi \Phi/\Phi_0)\cos(\theta) + d \sin(\pi \Phi/\Phi_0)\sin(\theta)]$

$\displaystyle H = \sum_n 4 E_C (n_g - n)^2 \left|n\right\rangle\left\langle n\right| - E_{J\Sigma}[\cos(\pi \Phi/\Phi_0)\frac{1}{2}\sum_n\left(\left|n+1\right\rangle\left\langle n\right| + \left|n\right\rangle\left\langle n+1\right| \right)+i d \sin(\pi \Phi/\Phi_0)\frac{1}{2}\sum_n\left(\left|n+1\right\rangle\left\langle n\right| - \left|n\right\rangle\left\langle n+1\right| \right)]$

where $E_C$ is the charge energy, $E_J$ is the Josephson energy, and $\left| n\right\rangle$ is the charge state with $n$ Cooper-pairs on the island that makes up the charge qubit.

### Qubit parameters

In [3]:
class SystemParameters:
    
    def __init__(self):
        # Input variable
        
        self.init_system()
        self.init_operators()
        self.init_pulse_operator()
        
    def init_system(self):
        self.initial_pulse = True
        self.state_num = 5
        self.alpha = 0.224574
        self.freq_ge = 3.9225 #GHz
        self.ens = np.array([ 2*np.pi*ii*(self.freq_ge - 0.5*(ii-1)*self.alpha) for ii in np.arange(self.state_num)])
        self.pts_per_period = 20
        self.dt = (1./self.freq_ge)/self.pts_per_period
        self.total_time=200.
        self.steps = int(self.total_time/self.dt)+1
        self.tlist = np.linspace(0,self.total_time,self.steps)

    def init_operators(self):
        # Create operator matrix in numpy array
        
        
        self.drive_freq = self.ens[1]
        
        x   = np.diag(np.sqrt(np.arange(1,self.state_num)),1)+np.diag(np.sqrt(np.arange(1,self.state_num)),-1)
        y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,self.state_num)),1)-np.diag(np.sqrt(np.arange(1,self.state_num)),-1))
        
        self.ops = [x,y]
        
        self.Hx_op = x
        self.Hy_op = y
        
        self.ops_len = len(self.ops)
        
        
    def gaussian(self,x, mu = 0. , sig = 1. ):
        return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    
    def init_pulse_operator(self, a=0.5):

        #functions
        def sin(t, a, f):
            return a*np.sin(2*np.pi*f*t)

        def cos(t, a, f):
            return a*np.cos(2*np.pi*f*t)

        # gaussian envelop
        gaussian_envelop = self.gaussian(np.linspace(-2,2,self.steps))
        
        # This is to generate a manual pulse
        manual_pulse = []

        manual_pulse.append(gaussian_envelop * cos(np.linspace(0,self.total_time,self.steps),a,self.freq_ge))
        manual_pulse.append(gaussian_envelop * sin(np.linspace(0,self.total_time,self.steps),a,self.freq_ge))
        

        self.manual_pulse = np.array(manual_pulse)
    

In [4]:
sys_para = SystemParameters()
H0 = Qobj(np.diag(sys_para.ens))

In [ ]:
class H_Pulse:
    def __init__(self):
        self.sys_para = SystemParameters()
        self.tlist = self.sys_para.tlist
        self.pulse_coeff_array = np.zeros(len(self.tlist))
        
    def gauss_span_length(self,length):
        return 6*length

    def square_span_length(self,length):
        ramp_sigma = 10
        return length + 6*ramp_sigma

    def gauss(self,tlist,time,amp,freq,length,phase):
        span_length = self.gauss_span_length(length)
        t0 = time+0.5*span_length
        return amp*np.exp(-(tlist-t0)**2/(2*length**2)) * (tlist>time) * (tlist< (time+span_length)) *\
        np.sin(2*np.pi*freq*tlist + phase)

    def add_pulse(self,time,amp,freq,length,phase,ptype):
        self.pulse_coeff_array += self.gauss(self.tlist,time,amp,freq,length,phase)
        return time + self.gauss_span_length(length)
        
    def get_coeff(self,time):
        index = self.time_to_index(time)
        return self.pulse_coeff_array[index]
    
    def time_to_index(self,time):
        return int(time/self.sys_para.dt - 1)

In [ ]:
# Define pulses
Hx_pulse = H_Pulse()
Hy_pulse = H_Pulse()

Hx_pulse.add_pulse(time=0,amp=0.03,freq=sys_para.ens[1]/(2*np.pi),length=20,phase=np.pi/2,ptype="gauss")
Hy_pulse.add_pulse(time=0,amp=0.03,freq=sys_para.ens[1]/(2*np.pi),length=20,phase=0,ptype="gauss")

fig, axes = plt.subplots(1, 1, figsize=(10,6))

plt.plot(Hx_pulse.tlist,Hx_pulse.pulse_coeff_array)
plt.plot(Hy_pulse.tlist,Hy_pulse.pulse_coeff_array)

axes.set_title('Input Pulse')

# Get Hamiltonian pulse value at time t
def Hx_coeff(t,args=None):
    Hx_t = Hx_pulse.get_coeff(t)
    return Hx_t

def Hy_coeff(t,args=None):
    Hy_t = Hy_pulse.get_coeff(t)
    return Hy_t

# Hamiltonian
Hx = Qobj(sys_para.Hx_op)
Hy = Qobj(sys_para.Hy_op)
H_t = [H0, [Hx,Hx_coeff], [Hy,Hy_coeff]]

# State operators, used for output
g_array= np.zeros(sys_para.state_num)
g_array[0] = 1
g_matrix = Qobj(np.diag(g_array))

e_array= np.zeros(sys_para.state_num)
e_array[1] = 1
e_matrix = Qobj(np.diag(e_array))

f_array= np.zeros(sys_para.state_num)
f_array[2] = 1
f_matrix = Qobj(np.diag(f_array))

h_array= np.zeros(sys_para.state_num)
h_array[3] = 1
h_matrix = Qobj(np.diag(h_array))

g = g_matrix
e = e_matrix
f = f_matrix
h = h_matrix

# Run Simulation
psi0 = g # start in ground cavity and excited transmon

output = mesolve(H_t, psi0, sys_para.tlist, [], [g, e, f, h],progress_bar=True)

# Plotting
g_out = output.expect[0]
e_out= output.expect[1]
f_out = output.expect[2]
h_out = output.expect[3]

fig, axes = plt.subplots(1, 1, figsize=(10,6))

axes.plot(sys_para.tlist, g_out, label="g")
axes.plot(sys_para.tlist, e_out, label="e")
axes.plot(sys_para.tlist, f_out, label="f")
axes.plot(sys_para.tlist, h_out, label="h")
axes.legend(loc=0)
axes.set_xlabel('Time')
axes.set_ylabel('Occupation probability')
axes.set_title('Rabi Oscillation')


10.0%. Run time:   3.92s. Est. time left: 00:00:00:35
20.0%. Run time:   9.42s. Est. time left: 00:00:00:37
30.0%. Run time:  17.09s. Est. time left: 00:00:00:39
40.0%. Run time:  25.61s. Est. time left: 00:00:00:38
50.0%. Run time:  33.18s. Est. time left: 00:00:00:33
60.0%. Run time:  38.34s. Est. time left: 00:00:00:25
70.0%. Run time:  39.71s. Est. time left: 00:00:00:17
80.0%. Run time:  41.07s. Est. time left: 00:00:00:10
